In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

# optimization
from scipy import optimize

# For quick OLS
#import statsmodels.formula.api as smf

# Import our toolbox
import clogit as clogit
import mlogit_ante as mlogit
import estimation as est

# Read in data

The dataset, `cars.csv`, contains cleaned and processed data. If you want to make changes, the notebook, `materialize.ipynb`, creates the data from the raw source datsets. 

In [3]:
cars = pd.read_csv('cars.csv')
lbl_vars = pd.read_csv('labels_variables.csv')
lbl_vals = pd.read_csv('labels_values.csv')

# convert from dataframe to dict
lbl_vals = {c: lbl_vals[c].dropna().to_dict() for c in lbl_vals.columns}

In [4]:
lbl_vars.set_index('variable', inplace=True)

## Overview of the dataset

In [5]:
lbl_vars.join(cars.mean(numeric_only=True).apply(lambda x: f'{x: .2f}').to_frame('Mean'))

label  \
variable                                                      
ye                         year (=first dimension of panel)   
ma                      market (=second dimension of panel)   
co                   model code (=third dimension of panel)   
zcode     alternative model code (predecessors and succe...   
brd                                              brand code   
type                                name of brand and model   
brand                                         name of brand   
model                                         name of model   
org       origin code (demand side, country with which c...   
loc       location code (production side, country where ...   
cla                                   class or segment code   
home      domestic car dummy (appropriate interaction of...   
frm                                               firm code   
qu                  sales (number of new car registrations)   
cy                  cylinder volume or displacement (in cc)   
hp                                       horsepower (in kW)   
we                                           weight (in kg)   
pl                   places (number, not reliable variable)   
do                    doors (number, not reliable variable)   
le                                           length (in cm)   
wi                                            width (in cm)   
he                                           height (in cm)   
li1       measure 1 for fuel efficiency (liter per km, a...   
li2       measure 2 for fuel efficiency (liter per km, a...   
li3       measure 3 for fuel efficiency (liter per km, a...   
li                average of li1, li2, li3 (used in papers)   
sp                                  maximum speed (km/hour)   
ac        time to acceleration (in seconds from 0 to 100...   
pr         price (in destination currency including V.A.T.)   
princ     =pr/(ngdp/pop): price relative to per capita i...   
eurpr     =pr/avdexr: price in common currency (in SDR t...   
exppr                 =pr/avexr: price in exporter currency   
avexr     av. exchange rate of exporter country (exporte...   
avdexr    av. exchange rate of destination country (dest...   
avcpr          av. consumer price index of exporter country   
avppr          av. producer price index of exporter country   
avdcpr      av. consumer price index of destination country   
avdppr      av. producer price index of destination country   
xexr                                           avdexr/avexr   
tax                                          percentage VAT   
pop                                              population   
ngdp      nominal gross domestic product of destination ...   
rgdp                            real gross domestic product   
engdp     =ngdp/avdexr: nominal gdp in common currency (...   
ergdp                                           =rgdp/avexr   
engdpc    =engdp/pop: nominal gdp per capita in common c...   
ergdpc                                           =ergdp/pop   
s                                market share (qu / qu_tot)   
qu_tot                      total sales in this market-year   
inc                                  avg. income per capita   

                         Mean  
variable                       
ye                      84.50  
ma                       3.00  
co                     207.50  
zcode                  177.76  
brd                     16.79  
type                      NaN  
brand                     NaN  
model                     NaN  
org                      2.72  
loc                      5.17  
cla                      2.30  
home                     0.32  
frm                     14.50  
qu                   35606.68  
cy                    1337.09  
hp                      50.10  
we                     934.49  
pl                       4.88  
do                       3.55  
le                     409.24  
wi                     163.44  
he                     140.46  
li1      

In [6]:
cars['s'].nunique()

5986

# Set up for analysis

In [7]:
price_var = 'princ'

In [8]:
cars['logp'] = np.log(cars[price_var])

In [9]:
# new variable: price elasticity heterogeneous for home-region 
cars['logp_x_home'] = cars[price_var] * cars['home']

### Dummy variables

For working with matrices, we want to have a column for each dummy variable. 

In [10]:
categorical_var = 'brand' # name of categorical variable
dummies = pd.get_dummies(cars[categorical_var]) # creates a matrix of dummies for each value of dummyvar
x_vars_dummies = list(dummies.columns[1:].values) # omit a reference category, here it is the first (hence columns[1:])

# add dummies to the dataframe 
assert dummies.columns[0] not in cars.columns, f'It looks like you have already added this dummy to the dataframe. Avoid duplicates! '
cars = pd.concat([cars,dummies], axis=1)

In [11]:
# NB! Let's take a look at the reference category
dummies.columns[0] #This is BMW
# Something might be going on 

'BMW'

### `x_vars`: List of regressors to be used 

In [12]:
x_vars = ['logp', 'home', 'cy', 'hp', 'we', 'li'] + ['tax', 'pop', 'inc'] #+ x_vars_dummies 
# If we include dummies for the brand, the 3-D matrix x becomes singular. Why?
# 
print(f'K = {len(x_vars)} variables selected.')

K = 9 variables selected.


In [13]:
cars = cars.dropna(axis=1)

In [14]:
cars['brand'].nunique() #Different car brands

33

In [15]:
cars['co'].nunique() #Different car models

285

In [17]:
K = len(x_vars) #The "household" characteristics
N = cars.ma.nunique() * cars.ye.nunique() #The market-year 'i'
J = 40 #The 40 different cars
x = cars[x_vars].values.reshape((N,J,K))

In [21]:
# Repeating the choice of 40 different cars in each market-year --> 6000 obs
cars['y'] = np.resize(np.arange(0,J), N*J)
y = cars['y'].values.reshape((N,J))

In [22]:
# Let's look at a particular 'market-year'

test = cars[(cars['ma']==1) & (cars['ye']==70)] 


# Interpretation: Each 'market-year', the top 40 most selling cars make up about 99.99% of all cars sold.
    # Each market-year share column (called "s" in the data) sums to one
# So in each year, let's define 40 different choices from j=0,...,40 
# NB! Remember that one particular car model may appear multiple times over different years and/or markets.
# This is slightly confusing -> basically, we model the 40 different choices, even though the same choice may appear again later

In [23]:
test

ye  ma   co  zcode  brd                               type      brand  \
0   70   1   15     14    2                       audi 100/200       audi   
1   70   1   26     35    4            citroen 2 CV 6 - 2 CV 4    citroen   
2   70   1   36     36    4                      citroen dyane    citroen   
3   70   1   64     67    7                           fiat 128       fiat   
4   70   1   71     80    8                        ford escort       ford   
5   70   1  134    159   16                   mercedes 200-300   mercedes   
6   70   1  165    197   19                        opel kadett       opel   
7   70   1  172    194   19                        opel rekord       opel   
8   70   1  186    202   20                        peugeot 504    peugeot   
9   70   1  187    207   20                        peugeot 304    peugeot   
10  70   1  196    227   22                          renault 4    renault   
11  70   1  212    337   22                         renault 16    renault   
12  70   1  213    338   22                          renault 6    renault   
13  70   1  214    319   22                         renault 12    renault   
14  70   1  217    242   36                         rover mini      rover   
15  70   1  269    284   29                     toyota corolla     toyota   
16  70   1  291    305   30  volkswagen coccinelle-beetle,1200         VW   
17  70   1  406     15    2                         audi 60-75       audi   
18  70   1  407    360    2               audi prinz 1000-1200       audi   
19  70   1  410    363    3            BMW 1502-1600-1602-1802        BMW   
20  70   1  412     17    3                           BMW 2002        BMW   
21  70   1  417     31    4                        citroen ami    citroen   
22  70   1  418    366    4                    citroen d-ds 20    citroen   
23  70   1  422    370   40                             daf 33        daf   
24  70   1  429    375    7                           fiat 124       fiat   
25  70   1  430    375    7                fiat 124 spec/sport       fiat   
26  70   1  431    376    7                      fiat 125-125S       fiat   
27  70   1  435    379    7                           fiat 850       fiat   
28  70   1  437     79    8                        ford 12-15M       ford   
29  70   1  439     78    8                        ford 20-26M       ford   
30  70   1  468    187   18                 nissan/datsun 1000     nissan   
31  70   1  488    418   19    opel/vxhall viva/firenze/magnum       opel   
32  70   1  490    420   20                        peugeot 204    peugeot   
33  70   1  491    421   20                        peugeot 404    peugeot   
34  70   1  497    426   22                       renault 8-10    renault   
35  70   1  521    446   46                  talbot/simca 1000  tal/simca   
36  70   1  524    448   46             talbot/simca 1301-1501  tal/simca   
37  70   1  536    301   30               volkswagen berl/1600         VW   
38  70   1  541    293   31                          volvo 140      volvo   
39  70   1  544    386    8                           ford 17M       ford   

        model  org  loc  ...  skoda  suzuki  tal/hillman  tal/matra  \
0     100/200    2    4  ...      0       0            0          0   
1        2CV6    1    3  ...      0       0            0          0   
2       dyane    1    3  ...      0       0            0          0   
3         128    3    5  ...      0       0            0          0   
4      escort    2    4  ...      0       0            0          0   
5         200    2    4  ...      0       0            0          0   
6       kadet    2    4  ...      0       0            0          0   
7      record    2    4  ...      0       0            0          0   
8         504    1    3  ...      0       0            0          0   
9         304    1    3  ...      0       0            0          0   
10          4    1    3  ...      0       0            0         

In [24]:
test['y'].nunique()

40

In [25]:
test['s'].sum()

0.9999999999999979

$$
u_{i j h}=\mathbf{x}_{i j} \boldsymbol{\beta}_o+\varepsilon_{i j h}, \quad j=1, \ldots, J
$$

where: 

- $i$ is the $\textit{country-year}$ pair
- $j$ is the alternative car
- $h$ is the household

First off: Are we: 

    1. interested in the marginal utility of a car's characteristic (conditional logit) or 
    
    2. the change in utility of car $j$ relative to car 1 given a change in household characteristics?

In this assignment, we are examning home bias - that is the propensity to choose a car manufactured in the home country. We are therefore interested in 1) and will use a conditional logit model.

In [26]:
x.shape

(150, 40, 9)

In [27]:
# The starting values to be passed to our optimizer
theta_start = clogit.starting_values(y, x)
theta_start.shape

(9,)

In [28]:
u = clogit.util(theta_start, x)
u.shape

(150, 40)

In [29]:
# Our conditional choice probabilities
# For coefficients ('theta') starting at zero, these must be equal to zero
# Intuition: No utility is gained by any car characteristics, thus market shares must be equal. Let's check this.
ccp = clogit.choice_prob(theta_start, x)
(ccp == 1/J) # all choice probs are equal to each other.

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [30]:
# In the first market-year, what are the choice-probs? (Given individuals place no weight on any car characteristics)
ccp.sum(axis=1) #Summing over the columns J equal one. Goods

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [31]:
print(ccp.shape)
np.unique(ccp) #Checking if we have any NaN

(150, 40)


array([0.025])

In [32]:
np.linalg.matrix_rank(x) # something is not right when you include the car-brand dummies -> rank-condition is not fulfilled -> matrix becomes singular
# Reference category is 'BMW'
# Could be that none of the top 40 cars sold in a given year and market is home-made and/or that they are BMW's -> some column(s) then become(s) zero.

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7], dtype=int64)

In [33]:
temp_df = pd.DataFrame(x[50]) #Looking at the 50th market-year
temp_df.columns=x_vars
temp_df.head()

logp  home      cy     hp      we   li   tax         pop         inc
0 -0.438650   0.0  1300.0   63.0   940.0  7.8  0.25  56730000.0  29513.0262
1 -0.226922   0.0  1800.0   66.0  1050.0  7.6  0.25  56730000.0  29513.0262
2 -0.143563   0.0  1600.0   75.0  1080.0  8.5  0.25  56730000.0  29513.0262
3  0.407682   0.0  2000.0  110.0  1425.0  9.8  0.25  56730000.0  29513.0262
4 -0.891653   1.0  1000.0   33.0   640.0  5.0  0.25  56730000.0  29513.0262

In [34]:
res = est.estimate(clogit.q, theta_start, y[:, 0], x)

Optimization terminated successfully.
         Current function value: 3.192185
         Iterations: 21
         Function evaluations: 270
         Gradient evaluations: 27


In [35]:
res['theta']

array([ 6.42855096e+00,  1.21097154e-01,  2.04601799e-04,  8.99830539e-03,
       -8.71445571e-03, -3.25885047e-01, -1.20239951e-06, -1.85070248e-07,
        0.00000000e+00])

In [36]:
theta_res = res['theta']

ccp_res = clogit.choice_prob(theta_res, x)

In [37]:
market_share = cars['s'].values.reshape((N,J))
market_share[0] # For the first market-year, what are the observed market shares of the 40 different cars?

array([0.01129646, 0.01464355, 0.02803195, 0.03263421, 0.03974679,
       0.02786459, 0.04393066, 0.0460226 , 0.02091936, 0.01025049,
       0.03598131, 0.02991469, 0.03347098, 0.01861823, 0.03075147,
       0.03911921, 0.05857422, 0.01255162, 0.00920452, 0.00878613,
       0.00920452, 0.01548033, 0.01945501, 0.03127445, 0.01799065,
       0.01380678, 0.00962291, 0.03012388, 0.03179743, 0.01045968,
       0.01108726, 0.02259291, 0.02322049, 0.02510324, 0.02656759,
       0.05899261, 0.01589872, 0.02091936, 0.03233297, 0.02175614])

In [38]:
ccp_res[0] # For the first market-year, what are the estimated market shares of the 40 different cars given our control variables?

array([0.03042623, 0.00955542, 0.01248148, 0.0094963 , 0.00351616,
       0.0139783 , 0.00733293, 0.00256075, 0.00382393, 0.01444181,
       0.00562273, 0.00490313, 0.00911352, 0.00927682, 0.01277412,
       0.017501  , 0.00455071, 0.01176699, 0.52052067, 0.03567286,
       0.08219593, 0.01358915, 0.00242163, 0.00813832, 0.00734331,
       0.01284128, 0.01010135, 0.00880175, 0.00637684, 0.01262221,
       0.03729414, 0.00764773, 0.01604368, 0.0044836 , 0.0073164 ,
       0.00343477, 0.00362545, 0.00516778, 0.00748987, 0.00374895])

In [39]:
market_share.shape

(150, 40)

In [40]:
ccp_res.shape

(150, 40)

---

### What does the log-likelhood contribution notation in the assignment mean??

In [ ]:
log_like_market = market_share * np.log(ccp)

In [88]:
log_like_market.shape

(150, 40)

In [89]:
log_like_market

array([[-0.04167127, -0.05401831, -0.10340648, ..., -0.07716901,
        -0.11927243, -0.08025577],
       [-0.06970409, -0.06428441, -0.09806126, ..., -0.04512321,
        -0.11367343, -0.09678501],
       [-0.03003248, -0.08001481, -0.06243417, ..., -0.05524576,
        -0.04033048, -0.10681655],
       ...,
       [-0.08623794, -0.03200978, -0.03263872, ..., -0.04278096,
        -0.06317241, -0.03111557],
       [-0.07847612, -0.03822282, -0.02797553, ..., -0.03170546,
        -0.07697698, -0.05334074],
       [-0.10661032, -0.03589541, -0.09121368, ..., -0.13260145,
        -0.04959585, -0.03861339]])

In [90]:
ccp.shape

(150, 40)

In [91]:
market_share.shape

(150, 40)

In [92]:
log_like_market = market_share * ccp_res

In [93]:
log_like_market.shape

(150, 40)

---

Old stuff

---

In [ ]:
#cars['y'] = np.resize(np.arange(0, M*T), N*J)
#y = cars['y'].values.reshape((N,J))

#cars['y'] = pd.factorize(cars['co'].values)[0]
#y = pd.factorize(cars['co'].values)[0]
#y = y.reshape((N,J))

#cars['brand']=pd.factorize(cars['brand'])[0]
#y = cars['co'].values.reshape((N,J))
#cars['co']=pd.factorize(cars['co'])[0]